In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached langchain_community-0.0.29-py3-none-any.whl.metadata (8.3 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached embedchain-0.1.104-py3-none-any.whl.metadata (10 kB)
  Using cached instructor-0.5.2-py3-none-any.whl.metadata (10 kB)
  Using cached opentelemetry_api-1.24.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.24.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.24.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached lancedb-0.5.7-py3-none-any.whl.metadata (17 kB)
  Using cached pyright-1.1.364-py3-none-any.whl.metadata (6.2 kB)
  Using cached pytest-8.2.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached p

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [32]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()
os.environ['OPENAI_API_KEY'] = "sk-proj-c6QyUoCJSg0Wv766Je2fT3BlbkFJSHvQiIuxGYc4MOtD9s3I"

In [56]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, DirectoryReadTool, FileReadTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### custom tools


In [57]:
# add this import for running in jupyter notebook
import nest_asyncio

nest_asyncio.apply()

In [58]:
from crewai_tools import BaseTool
import json
from bson import ObjectId
from langchain_community.document_loaders.mongodb import MongodbLoader
# from langchain.chat_models import ChatOpenAI
# from langchain.agents import create_json_agent
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.tools.json.tool import  JsonSpec
import json
class ProjectManagementTool(BaseTool):
    name: str ="Project Management Tool Analysis"
    description: str = ("Analyzes the questation of given text "
         "fetch the data from the Mongo DB and and try to answer the given questation based of the Mongo DB result and strictly do not add any new things by own")
    def get_data(self):
        loader = MongodbLoader(
        connection_string="mongodb://localhost:27017/",
        db_name="jira_db",
        collection_name="jira_issues",
        )
        docs = loader.load()
        json_data_list = {"dummy":[]}

        # Assume 'docs' is a list of objects that contain the 'page_content' attribute
        for doc in docs:
            data = doc.page_content.replace("'", '"')
            # Convert ObjectId to string and None to null
            data = data.replace('ObjectId(', '').replace(')', '').replace('None', 'null')
        
            # Convert the string to a JSON object
            try:
                data_json = json.loads(data)
                json_data_list['dummy'].append(data_json)  # Append the dictionary to the list
            except json.JSONDecodeError as e:
                print(f"An error occurred: {e}")
        
        # Write the list of JSON objects to a file
        # with open('output3.json', 'w', encoding='utf-8') as f:
        #     json.dump(json_data_list, f, ensure_ascii=False, indent=4)
        return json_data_list


    def _run(self, text: str) -> str:
        data = self.get_data()
        # json_data_list = self.get_data()
        # spec=JsonSpec(dict_=json_data_list,max_value_length=4000)
        # json_tool=JsonToolkit(spec=spec)
        return data

        

        








In [59]:
project_management_tool = ProjectManagementTool()

In [60]:
def get_data():
    loader = MongodbLoader(
    connection_string="mongodb://localhost:27017/",
    db_name="jira_db",
    collection_name="jira_issues",
    )
    docs = loader.load()
    json_data_list = {"dummy":[]}

    # Assume 'docs' is a list of objects that contain the 'page_content' attribute
    for doc in docs:
        data = doc.page_content.replace("'", '"')
        # Convert ObjectId to string and None to null
        data = data.replace('ObjectId(', '').replace(')', '').replace('None', 'null')
    
        # Convert the string to a JSON object
        try:
            data_json = json.loads(data)
            json_data_list['dummy'].append(data_json)  # Append the dictionary to the list
        except json.JSONDecodeError as e:
            print(f"An error occurred: {e}")
    
    # Write the list of JSON objects to a file
    with open('./instructions/output.json', 'w', encoding='utf-8') as f:
        json.dump(json_data_list, f, ensure_ascii=False, indent=4)
    return json_data_list

In [82]:
!cat ./instructions/output.json

{
    "dummy": [
        {
            "_id": "664f1e709ba2334379c1740a",
            "Issue Key": "IN-2",
            "Summary": "Build JIRA Integration API",
            "Description": null,
            "Status": "To Do",
            "Assignee": "Unassigned",
            "Comments": [
                {
                    "Author": "data science",
                    "Created": "2024-05-21T21:11:57.437+0530",
                    "Comment": "this is for testing…"
                }
            ]
        },
        {
            "_id": "664f1e709ba2334379c1740b",
            "Issue Key": "IN-1",
            "Summary": "jira integration",
            "Description": "build the system which can take the Jira issues and develop the code by understanding the summary.",
            "Status": "To Do",
            "Assignee": "data science",
            "Comments": [
                {
                    "Author": "data science",
                    "Created": "2024-05-20T22:54:37.247+0530",
  

In [61]:
json_data_list = get_data()

directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()


In [62]:
project_manager_agent = Agent(
    role="Project Manager",
    goal="Monitor and analyze jira issues"
         "to identify which ticket is has been assigned whome complexity of the tasks",
    backstory="Specializing in Project Management, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Poject Manager Agent is the cornerstone for "
              "informing issues and comments decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [directory_read_tool,file_read_tool]
)

### Create The Task

In [69]:
# Task for Data Analyst Agent: Analyze Market Data
project_management_task = Task(
    description=(
        "Continuously monitor and analyze jira issues"
        "and try to analyse the {questation} find out the relevent answer from jira issues json data"
        "Use generative model to "
        "prepare the right answer"
    ),
    expected_output=(
        "give the relevent answer if you know"
        "otherwise just say I don't know the answer of your questation {questation}."
    ),
    agent=project_manager_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [70]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
project_manager_crew = Crew(
    agents=[project_manager_agent],
    
    tasks=[project_management_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    # process=Process.hierarchical,
    verbose=True,
    memory=True
    
)

2024-05-23 17:53:56,169 - 8008670912 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [88]:
inputs = {
    "questation": "How many Tickets are in JIRA"
}


In [87]:
result = project_manager_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Project Manager
 [INFO]: == Starting Task: Continuously monitor and analyze jira issuesand try to analyse the How many Tickets are in JIRA find out the relevent answer from jira issues json dataUse generative model to prepare the right answer


> Entering new CrewAgentExecutor chain...
I have identified the file containing the Jira issues data, and I need to read its content to analyze the information.

Action: Read a file's content
Action Input: {"file_path": "./jira_data/output.json"}
 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './jira_data/output.json'.
 Tool Read a file's content accepts these inputs: Read a file's content(file_path: 'string') - A tool that can be used to read a file's content.

Thought: I need to list the files in the directory to see if there's a different file containing the Jira issues data.
Action: List files in directory
Action Input: {"directory_path": "./jira_da

In [89]:
result

'There are 2 tickets in JIRA.'

In [78]:
import gradio as gr
def bot(que):
    inputs = {
    "questation": que
    }
    result = project_manager_crew.kickoff(inputs=inputs)
    return result





In [80]:
import gradio as gr

# This is the function that will process the input text and return a response
def chatbot_function(input_text):
    # Imagine this function processes the input_text and generates a response
    inputs = {"questation": que}
    result = project_manager_crew.kickoff(inputs=inputs)
    return result

# Create the Gradio interface
iface = gr.Interface(
    fn=chatbot_function,  # the function the interface will interact with
    inputs=gr.Textbox(lines=2, placeholder="Type something here..."),  # the input component
    outputs=gr.Textbox(),  # the output component
    title="Project management Agent",
    description="Type your message and get a response from the chatbot."
)

# Launch the application
iface.launch()

Exception in thread Thread-15 (run):
Traceback (most recent call last):
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/

IMPORTANT: You are using gradio version 4.25.0, however version 4.29.0 is available, please upgrade.
--------


Exception in thread Thread-16 (run):
Traceback (most recent call last):
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/

KeyboardInterrupt: 

In [81]:
import gradio as gr

def chatbot_function(input_text):
    response = f"You said: {input_text}"
    return response

iface = gr.Interface(
    fn=chatbot_function,
    inputs=gr.Textbox(lines=2, placeholder="Type something here..."),
    outputs=gr.Textbox(),
    title="Project Management Agent",
    description="Type your message and get a response from the chatbot."
)

iface.launch()

Exception in thread Thread-22 (run):
Traceback (most recent call last):
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/

IMPORTANT: You are using gradio version 4.25.0, however version 4.29.0 is available, please upgrade.
--------


Exception in thread Thread-23 (run):
Traceback (most recent call last):
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/site-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rajesh.kumar3/miniconda3/envs/sql_query/lib/python3.11/

KeyboardInterrupt: 

In [ ]:
import asyncio
import threading
import gradio as gr

def run_gradio():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    def chatbot_function(input_text):
        response = f"You said: {input_text}"
        return response

    iface = gr.Interface(
        fn=chatbot_function,
        inputs="text",
        outputs="text",
        title="Project Management Agent",
        description="Type your message and get a response from the chatbot."
    )

    iface.launch()

t = threading.Thread(target=run_gradio)
t.start()